In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, DatasetDict, Dataset
from datasets import Dataset as HFDataset
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

In [134]:
ds_train = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/LLM-Sentiment-Analysis-lhl/csv_files/train.csv')
ds_test = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/LLM-Sentiment-Analysis-lhl/csv_files/test.csv')

In [135]:
# Function to remove punctuation from the text col
def remove_punctuation(text):
    text = "".join([char for char in text if char not in string.punctuation])

    return text

# Run the function on the datasets
ds_train['text_tokenized'] = ds_train['text_tokenized'].apply(lambda x: remove_punctuation(x))
ds_test['text_tokenized'] = ds_test['text_tokenized'].apply(lambda x: remove_punctuation(x))

In [ ]:
ds_train.head()

: 

In [ ]:
# # Clean and join the tokens in 'text_tokenized' into a single string for each document
# ds_train = ds_train['text_tokenized'].apply(
#     lambda tokens: " ".join(token.strip() for token in tokens if token not in string.punctuation)
# )
# ds_test = ds_test['text_tokenized'].apply(
#     lambda tokens: " ".join(token.strip() for token in tokens if token not in string.punctuation)
# )

# # Check a few rows to verify the fix
# print(ds_train.head())
# print(ds_test.head())

: 

In [136]:
# Instantiate the PorterStemmer and WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [137]:
from nltk.stem import PorterStemmer

# Define the stemming function
def stem_text(text):
    stemmer = PorterStemmer()
    # Ensure the input is a string and split it into words
    stemmed_words = [stemmer.stem(word) for word in text.split()]
    return " ".join(stemmed_words)

# Apply stemming to the 'text_tokenized_str' column in the dataset
ds_train['text_stemmed'] = ds_train['text_tokenized'].apply(stem_text)
ds_test['text_stemmed'] = ds_test['text_tokenized'].apply(stem_text)

# Check the first few rows to confirm the stemming
print(ds_train['text_stemmed'].head())
print(ds_test['text_stemmed'].head())

0    i rent i am curiousyellow from my video store ...
1    i am curiou yellow is a risibl and pretenti st...
2    if onli to avoid make thi type of film in the ...
3    thi film wa probabl inspir by godard masculin ...
4    oh brotheraft hear about thi ridicul film for ...
Name: text_stemmed, dtype: object
0    i love scifi and am will to put up with a lot ...
1    worth the entertain valu of a rental especi if...
2    it a total averag film with a few semialright ...
3    star rate saturday night friday night friday m...
4    first off let me say if you havent enjoy a van...
Name: text_stemmed, dtype: object


In [138]:
from nltk.stem import WordNetLemmatizer

# Define the lemmatization function
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # Ensure the input is a string and split it into words
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in text.split()]
    return " ".join(lemmatized_words)

# Apply lemmatization to the 'text_tokenized_str' column in the dataset
ds_train['text_lemmatized'] = ds_train['text_tokenized'].apply(lemmatize_text)
ds_test['text_lemmatized'] = ds_test['text_tokenized'].apply(lemmatize_text)

# Check the first few rows to confirm the lemmatization
print(ds_train['text_lemmatized'].head())
print(ds_test['text_lemmatized'].head())

0    i rent i be curiousyellow from my video store ...
1    i be curious yellow be a risible and pretentio...
2    if only to avoid make this type of film in the...
3    this film be probably inspire by godards mascu...
4    oh brotherafter hear about this ridiculous fil...
Name: text_lemmatized, dtype: object
0    i love scifi and be will to put up with a lot ...
1    worth the entertainment value of a rental espe...
2    its a totally average film with a few semialri...
3    star rat saturday night friday night friday mo...
4    first off let me say if you havent enjoy a van...
Name: text_lemmatized, dtype: object


In [ ]:
# from nltk.corpus import stopwords

# # Load NLTK's list of stop words
# stop_words = set(stopwords.words('english'))

# # Check for rows with only stop words or empty strings
# def contains_only_stopwords_or_empty(text):
#     # Split the text into words
#     words = text.split()
#     # Check if all words are stop words or if the text is empty
#     return all(word in stop_words for word in words) or len(words) == 0

# # Apply the function to the 'text_lemmatized' column
# ds_train['only_stopwords_or_empty'] = ds_train['text_lemmatized'].apply(contains_only_stopwords_or_empty)
# ds_test['only_stopwords_or_empty'] = ds_test['text_lemmatized'].apply(contains_only_stopwords_or_empty)

# # Filter rows where the column contains only stop words or is empty
# train_filtered = ds_train[ds_train['only_stopwords_or_empty']]
# test_filtered = ds_test[ds_test['only_stopwords_or_empty']]

# # Print rows with only stop words or empty strings
# print("Rows in training set with only stop words or empty strings:")
# print(train_filtered)

# print("\nRows in test set with only stop words or empty strings:")
# print(test_filtered)

: 

In [ ]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Join the tokens in 'text_lemmatized' into a single string for each document
ds_train['text_lemmatized'] = ds_train['text_lemmatized'].apply(lambda tokens: ' '.join(tokens))
ds_test['text_lemmatized'] = ds_test['text_lemmatized'].apply(lambda tokens: ' '.join(tokens))

# Fit and transform the 'text_lemmatized_str' column for the training dataset
tfidf_train_matrix = tfidf_vectorizer.fit_transform(ds_train['text_lemmatized'])

# Transform the 'text_lemmatized_str' column for the test dataset
tfidf_test_matrix = tfidf_vectorizer.transform(ds_test['text_lemmatized'])

# Convert the training matrix to a DataFrame for better readability (optional)
import pandas as pd
tfidf_train_df = pd.DataFrame(
    tfidf_train_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

# Convert the test matrix to a DataFrame for better readability (optional)
tfidf_test_df = pd.DataFrame(
    tfidf_test_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out()
)

# Display the resulting DataFrames (optional)
print("TF-IDF Training DataFrame:")
print(tfidf_train_df.head())

print("\nTF-IDF Test DataFrame:")
print(tfidf_test_df.head())

: 

: 